# Imports

In [ ]:
import numpy as np
import pandas as pd
import zarr
import dask
from dask import delayed
from dask.distributed import Client, LocalCluster, progress
from dask_jobqueue import SLURMCluster
import streamz
import holoviews as hv
from holoviews.streams import Stream, param
from holoviews.operation.datashader import regrid
from bokeh.models.tools import HoverTool
import matplotlib.pyplot as plt
import qgrid
import ipywidgets as widgets
from tqdm import tnrange, tqdm, tqdm_notebook
import warnings
from functools import partial
from cytoolz import compose, get_in
from operator import getitem
import nd2reader
from importlib import reload
import traceback
import hvplot.pandas
import param
import parambokeh
from traitlets import All
import cachetools
from collections import namedtuple, defaultdict
import skimage.morphology
import scipy

IDX = pd.IndexSlice

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# from processing import *
# from trench_detection import *
# from trench_segmentation import *
# from trench_segmentation.watershed import *
# from util import *
# from ui import *
import common, trench_detection, util
import ui, diagnostics, metadata
import workflow, image, geometry
import trench_detection.hough, trench_detection.core
import trench_segmentation.watershed

In [ ]:
%load_ext line_profiler
hv.extension("bokeh")
%matplotlib inline
tqdm.monitor_interval = 0

# Restore data

In [ ]:
%store -r trench_points trench_diag

# Config

In [ ]:
cluster = SLURMCluster(
    queue="short",
    walltime="2:00:00",
    # job_extra=['-p transfer'],
    # job_extra=['--cores-per-socket=8'],
    # interface='ib0',
    memory="16GB",
    local_directory="/tmp",
    cores=1,
    processes=1,
    # diagnostics_port=('127.0.0.1', 8787),
    env_extra=['export PYTHONPATH="/home/jqs1/projects/matriarch"'],
)
client = Client(cluster)

In [ ]:
cluster._widget().children[1].children[1].children[0].children[0].layout.width = "200px"
cluster

In [ ]:
# client = Client()

In [ ]:
cluster.stop_workers(cluster.jobs)

# Loading data

In [ ]:
# nd2_filenames = ['/n/scratch2/jqs1/fidelity/all/180405_txnerr.nd2', '/n/scratch2/jqs1/fidelity/all/180405_txnerr001.nd2']
# nd2_filenames = ['/n/scratch2/jqs1/fidelity/all/180405_txnerr002.nd2']#, '/n/scratch2/jqs1/fidelity/all/TrErr002_Exp.nd2']
# nd2_filenames = ['/n/scratch2/jqs1/fidelity/all/TrErr002_Exp.nd2']
nd2_filenames = [
    "/n/scratch2/jqs1/fidelity/all/180405_txnerr.nd2",
    "/n/scratch2/jqs1/fidelity/all/180405_txnerr001.nd2",
    "/n/scratch2/jqs1/fidelity/all/180405_txnerr002.nd2",
    "/n/scratch2/jqs1/fidelity/all/TrErr002_Exp.nd2",
]
# nd2_filenames = ['/home/jqs1/scratch/fidelity/180518_triplegrowthcurve/PHASE_GC001.nd2', '/home/jqs1/scratch/fidelity/180518_triplegrowthcurve/PHASE_GC002.nd2']

In [ ]:
all_frames, metadata, parsed_metadata = workflow.get_nd2_frame_list(nd2_filenames)

# Reload

In [ ]:
def do_reload():
    from importlib import reload
    import util, trench_detection, diagnostics, workflow, image

    reload(util)
    reload(trench_detection)
    reload(diagnostics)
    reload(workflow)
    reload(image)


client.run(do_reload)
do_reload()

# Finding trenches

In [ ]:
frames_to_process = all_frames.loc[IDX[:, :, ["MCHERRY"], :10], :]

In [ ]:
len(frames_to_process)

## Frame quality finding

In [ ]:
radial_psd2 = compose(image.radial_profile, image.psd2)
frame_psd2s_futures = {
    idx: client.submit(
        radial_psd2, client.submit(workflow._get_nd2_frame, **idx._asdict())
    )
    for idx, row in util.iter_index(frames_to_process)
}

In [ ]:
frame_psd2s = client.gather(frame_psd2s_futures)

In [ ]:
FrameStream = ui.DataframeStream.define(
    "FrameStream", frames_to_process.index.to_frame(index=False)
)
frame_stream = FrameStream()

box = ui.dataframe_browser(frame_stream)
frame_stream.event()
box

In [ ]:
%%opts Layout [shared_axes=False]
dict_viewer(
    frame_psd2s, frame_stream, wrapper=lambda k, v: hv.Curve(np.log(v))
) + ui.image_viewer(frame_stream)

## Run trench finding

In [ ]:
# locally: get trench_points dict?? (how to organize? use dict proxy to index into it?)
# where do I list all trenches, so that I can map over them?? e.g., compute per-timepoint focus
# turn trench_points into df
# locally: get diag df (by trench_set)
# then dask

In [ ]:
# trench_points_futures = {idx: client.submit(get_trenches,
#                                            client.submit(workflow._get_nd2_frame, **idx._asdict())) for idx, row in util.iter_index(frames_to_process)}

In [ ]:
find_trenches_diag = diagnostics.wrap_diagnostics(
    trench_detection.hough.find_trenches, ignore_exceptions=True, pandas=True
)
trench_info_futures = {
    idx: client.submit(
        find_trenches_diag, client.submit(workflow.get_nd2_frame, **idx._asdict())
    )
    for idx, row in util.iter_index(frames_to_process)
}

In [ ]:
client.cancel(trench_info_futures)

In [ ]:
%%time
trench_info = util.apply_map_futures(
    client.gather, trench_info_futures, predicate=lambda x: x.status == "finished"
)

In [ ]:
len(trench_info)

In [ ]:
errs = {k: v[2] for k, v in trench_info.items() if v[2] is not None}
errs

In [ ]:
%%time
trench_points, trench_diag, trench_err = workflow.unzip_trench_info(trench_info)

In [ ]:
trench_points2 = trench_points.copy()

In [ ]:
len(trench_points)

In [ ]:
%%time
%store trench_points
%store trench_diag

## Analysis

In [ ]:
trench_diag.tail()

In [ ]:
bad_angle = trench_diag["find_trench_lines.hough_2.angle"].abs() > 2
bad_angle.sum()

In [ ]:
bad_pitch = (trench_diag["find_trench_lines.hough_2.pitch"] - 24).abs() > 1
bad_pitch.sum()

In [ ]:
selected = trench_diag[bad_pitch]  # trench_diag[bad_angle | bad_period]

In [ ]:
%%time
trench_points_good = trench_points[~util.multi_join(trench_points.index, bad_pitch)]

In [ ]:
trench_points_good.head()

In [ ]:
(len(trench_points_good), len(trench_points_good) / len(trench_points))

In [ ]:
frame_stream.event(_df=selected.index.to_frame(index=False))

# Trench finding QA

In [ ]:
FrameStream = ui.DataframeStream.define(
    "FrameStream", selected.index.to_frame(index=False)
)
frame_stream = FrameStream()

box = ui.dataframe_browser(frame_stream)
frame_stream.event()
box

In [ ]:
ui.image_viewer(frame_stream)

In [ ]:
ui.show_frame_info(trench_diag, frame_stream)

In [ ]:
g = ui.show_grid(selected, stream=frame_stream)
g

In [ ]:
%%time
frame = workflow.get_nd2_frame(**dict(frame_stream.get_param_values()))
_, diag, _ = diagnostics.wrap_diagnostics(trench_detection.hough.find_trenches)(frame)

In [ ]:
frame.shape

In [ ]:
a = get_in("label_2.find_trench_lines.hough_1.log_hough".split("."), diag)

In [ ]:
b = get_in("label_2.find_trench_lines.hough_1.input".split("."), diag).data

In [ ]:
d = (
    get_in("label_2.find_trench_lines.hough_1.profile".split("."), diag)
    .Curve.I.data["y"]
    .values
)

In [ ]:
plt.plot(d)

In [ ]:
fs, Pxx = scipy.signal.periodogram(np.trim_zeros(d))

In [ ]:
plt.plot(Pxx)

In [ ]:
b.max()

In [ ]:
c = image.hough_line_intensity(b)

In [ ]:
(~np.isnan(a.data)).sum()

In [ ]:
ui.show_plot_browser(diag)

# Segmentation

In [ ]:
selected_trenches = trench_points_good.loc[
    IDX["/n/scratch2/jqs1/fidelity/all/TrErr002_Exp.nd2", :3, ["MCHERRY"], 0, :, :], :
]

In [ ]:
%%time
selected_trench_bboxes = workflow.get_trench_bboxes(
    selected_trenches, workflow.get_filename_image_limits(metadata)
)

In [ ]:
%%time
selected_trenches = pd.concat([selected_trenches, selected_trench_bboxes], axis=1)

In [ ]:
%%time
trench_stacks = workflow.get_trench_stacks(
    selected_trenches, all_frames, workflow.get_filename_image_limits(metadata)
)

In [ ]:
from trench_segmentation.watershed import _trench_img

In [ ]:
list(trench_stacks.keys())[224]

In [ ]:
img_stack = list(trench_stacks.values())[624]

In [ ]:
trench_seg, trench_seg_diag, _ = diagnostics.wrap_diagnostics(
    trench_segmentation.watershed.segment_trench
)(img_stack[91])

In [ ]:
ui.show_plot_browser(trench_seg_diag)

In [ ]:
%%time
segs, seg_diags, seg_errs = zip(
    *[
        diagnostics.wrap_diagnostics(trench_segmentation.watershed.segment_trench)(img)
        for img in img_stack
    ]
)

In [ ]:
%%opts Image [width=120,height=400]
%%opts Layout [normalize=False]
(
    hv.HoloMap({t: hv.Image(img) for t, img in enumerate(img_stack)})
    + hv.HoloMap({t: hv.Image(img) for t, img in enumerate(segs)})
)  # .cols(1)

In [ ]:
ui.show_plot_browser(seg_diags[62])

In [ ]:
img = img_stack[91].T

In [ ]:
plt.imshow(img)

In [ ]:
img_blurred = skimage.filters.gaussian(img, 0.5)

In [ ]:
plt.imshow(img_blurred)

In [ ]:
img_k1, img_k2 = image.hessian_eigenvalues(img_blurred)

In [ ]:
plt.imshow(img_k1)

In [ ]:
img_k1_frangi = skimage.filters.frangi(img_k1, scale_range=(1, 3), scale_step=0.5)

In [ ]:
ui.RevImage(img_k1_frangi)

In [ ]:
plt.imshow(img_k1_frangi)

In [ ]:
img_k1_thresh = img_k1_frangi > skimage.filters.threshold_otsu(img_k1_frangi)

In [ ]:
plt.imshow(img_k1_thresh)

In [ ]:
plt.imshow(
    skimage.filters.roberts_pos_diag(img_k1) - skimage.filters.roberts_neg_diag(img_k1)
)

In [ ]:
plt.imshow(img_k2)

In [ ]:
img_thresh = img_blurred > skimage.filters.threshold_otsu(img_blurred)

In [ ]:
plt.imshow(img_thresh)

In [ ]:
img_edt = scipy.ndimage.distance_transform_edt(img_thresh)

In [ ]:
plt.imshow(img_edt)

# Old

In [ ]:
get_trenches(root_group["raw"][str(pos)][1, 30], diagnostics=diag_pos[pos])

In [ ]:
diag = tree()
_ = get_trenches(root_group["raw"][str(pos)][0, 1], diagnostics=diag)

In [ ]:
def f(img_stack):
    ary = np.stack(
        [
            segment_trench(img_stack[t], diagnostics=None)
            for t in range(img_stack.shape[0])
        ],
        axis=0,
    )
    ary = zarr.array(ary, compressor=DEFAULT_FRAME_COMPRESSOR)
    return ary


trench_seg_masks = positionwise_trenchwise_map(
    root_group["raw"],
    trench_points_pos,
    f,
    channel_slice=1,
    preload=True,
    time_slice=slice(None),
    positions=range(1),
)

In [ ]:
def f(img_stack):
    return pd.Series(np.percentile(img_stack, 95, axis=(1, 2)))
    # return pd.Series(np.max(img_stack, axis=(1,2)))


trench_traces_all = positionwise_trenchwise_map(
    root_group["raw"],
    trench_points_pos,
    f,
    channel_slice=2,
    preload=True,
    time_slice=slice(None),
    positions=range(100),
)